In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pycaret.time_series import *

In [3]:
df = pd.read_csv('data_preprocess\export-pm25_eng-1h.csv')

In [4]:
df.head()

,Unnamed: 0,timestamp,timezone,humidity,pm_10,pm_2_5,pm_2_5_sp,temperature
0,0,2023-03-31 22:00:00,Asia/Bangkok,62.659563,4.727273,4.727273,4.727273,27.147784
1,1,2023-04-02 18:00:00,Asia/Bangkok,69.995698,7.857143,7.142857,7.142857,32.246966
2,2,2023-04-02 19:00:00,Asia/Bangkok,65.608215,8.400000,7.800000,7.800000,32.518829
3,3,2023-04-02 20:00:00,Asia/Bangkok,65.489705,8.750000,8.266667,8.266667,31.864227
4,4,2023-04-02 21:00:00,Asia/Bangkok,65.811462,10.316667,9.083333,9.083333,31.412888


In [5]:
df.drop(columns=['Unnamed: 0','pm_10','pm_2_5_sp','timezone'], inplace=True)


In [104]:
df

,timestamp,humidity,pm_2_5,temperature
0,2023-03-31 22:00:00,62.659563,4.727273,27.147784
1,2023-04-02 18:00:00,69.995698,7.142857,32.246966
2,2023-04-02 19:00:00,65.608215,7.800000,32.518829
3,2023-04-02 20:00:00,65.489705,8.266667,31.864227
4,2023-04-02 21:00:00,65.811462,9.083333,31.412888
...,...,...,...,...
14791,2025-02-11 08:00:00,90.929747,37.568627,23.684319
14792,2025-02-11 09:00:00,83.511569,38.800000,25.896389
14793,2025-02-11 10:00:00,72.046736,23.333333,28.852626
14794,2025-02-11 11:00:00,64.332405,21.850000,30.593716


In [6]:
df.isna().sum()

timestamp      0
humidity       4
pm_2_5         0
temperature    4
dtype: int64

In [7]:
df['temperature'] = df['temperature'].fillna(df['temperature'].mean())
df['humidity'] = df['humidity'].fillna(df['humidity'].mean())
df.isna().sum()

timestamp      0
humidity       0
pm_2_5         0
temperature    0
dtype: int64

In [8]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [9]:
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['month'] = df['timestamp'].dt.month

In [109]:
df['pm_2_5_lag1'] = df['pm_2_5'].shift(1)
df['pm_2_5_lag7'] = df['pm_2_5'].shift(7)

In [110]:
from scipy import stats
df = df[(np.abs(stats.zscore(df['humidity'])) < 3)]  # ลบค่าผิดปกติออก
df = df[(np.abs(stats.zscore(df['temperature'])) < 3)]  # ลบค่าผิดปกติออก
df = df[(np.abs(stats.zscore(df['pm_2_5'])) < 3)]  # ลบค่าผิดปกติออก

In [111]:
df = df.sort_values('timestamp')
df.set_index('timestamp', inplace=True)

In [112]:
df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek

In [113]:
df = df.dropna()

In [114]:
df

,humidity,pm_2_5,temperature,hour,day_of_week,month,pm_2_5_lag1,pm_2_5_lag7
timestamp,,,,,,,,
2023-04-03 00:00:00,66.876119,7.200000,30.042084,0,0,4,7.466667,4.727273
2023-04-03 01:00:00,67.430725,7.783333,29.392181,1,0,4,7.200000,7.142857
2023-04-03 02:00:00,68.017985,16.500000,29.118591,2,0,4,7.783333,7.800000
2023-04-03 03:00:00,68.853455,24.783333,28.612701,3,0,4,16.500000,8.266667
2023-04-03 04:00:00,69.691119,28.543860,28.313422,4,0,4,24.783333,9.083333
...,...,...,...,...,...,...,...,...
2025-02-11 08:00:00,90.929747,37.568627,23.684319,8,1,2,31.325581,25.416667
2025-02-11 09:00:00,83.511569,38.800000,25.896389,9,1,2,37.568627,26.355932
2025-02-11 10:00:00,72.046736,23.333333,28.852626,10,1,2,38.800000,27.033333


In [115]:
df = df.asfreq('D')  # 'D' คือความถี่รายวัน (Daily), สามารถเปลี่ยนเป็น 'H' สำหรับรายชั่วโมงหรือความถี่อื่น ๆ

In [116]:
X = df[['pm_2_5_lag1', 'pm_2_5_lag7', 'hour', 'day_of_week', 'humidity', 'temperature', 'pm_2_5']]

In [118]:
s = setup(
        data = X,
        target='pm_2_5',  # target variable
        fh = 7,  # forecast horizon (ทำนาย 7 วัน)
        seasonal_period = 12,
        fold = 5,  # cross-validation
        session_id = 123,  # เพื่อความสามารถในการทำซ้ำ
        numeric_imputation_target='mean',
        numeric_imputation_exogenous='mean')  # กำหนดการ transform target เพื่อปรับค่าให้เหมาะสม

,Description,Value
0,session_id,123
1,Target,pm_2_5
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(681, 7)"
5,Transformed data shape,"(681, 7)"
6,Transformed train set shape,"(674, 7)"
7,Transformed test set shape,"(7, 7)"
8,Rows with missing values,16.2%
9,Fold Generator,ExpandingWindowSplitter


In [120]:
best_model = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:47:20
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Auto ARIMA


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
arima,ARIMA,0.0001,0.0001,0.0008,0.0008,0.0000,0.0000,-78104088109445504040960.0000,0.1420


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

KeyboardInterrupt: 